# [Research Task - Create visuals for PUC 99314.11 leg report](https://github.com/cal-itp/data-analyses/issues/1656)
1. line graph of each metric (UPT, VRM, PMT) by agency
- x-axis is year
- y-axis is metric
- each line is an agency
- dotted line is average metric for all agencies in the year

2. line graph of each metric, by district
- similar to above
- each line is a district
- dotted line is average metrics for all districts the year

3. line graph of each metric, by mode
- similar to above
- each line is a mode
- dotter line is average metric for all modes in the year

Maybe try a box plot to show min/max/average for each metric?

In [1]:
import pandas as pd
from calitp_data_analysis.sql import to_snakecase
import altair as alt

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [3]:
gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"
ntd_name = "ntd_operator_data_18_23.parquet"

ntd_all_metrics = pd.read_parquet(f"{gcs_path}{ntd_name}")
ntd_all_metrics.info()

_request non-retriable exception: ('Error code invalid_grant: Refresh token has expired', '{"error":"invalid_grant","error_description":"Refresh token has expired"}')
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/gcsfs/retry.py", line 135, in retry_request
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/gcsfs/core.py", line 467, in _request
    headers=self._get_headers(headers),
            ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/gcsfs/core.py", line 444, in _get_headers
    self.credentials.apply(out)
  File "/opt/conda/lib/python3.11/site-packages/gcsfs/credentials.py", line 223, in apply
    self.maybe_refresh()
  File "/opt/conda/lib/python3.11/site-packages/gcsfs/credentials.py", line 211, in maybe_refresh
    self.credentials.refresh(req)
  File "/opt/conda/lib/python3.11/site-packages/google/auth/external_account_authorized_user.p

OAuthError: ('Error code invalid_grant: Refresh token has expired', '{"error":"invalid_grant","error_description":"Refresh token has expired"}')